# 쿼리 추출기 (Query Extractor)

한국어 음악 검색 쿼리를 CLAP/MuQ-MuLan 모델에 사용할 수 있는 영어 템플릿 형식으로 변환합니다.

### 템플릿 형식
```
{genre}, {mood} mood, {tempo or energy}, 
{main instruments}, {vocal info + language}, 
suitable for {use-case or scene}
```

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv("../.env.local", override=True)

# ANTHROPIC_API_KEY 가 설정되어 있는지 확인
if not os.getenv("ANTHROPIC_API_KEY"):
    raise ValueError("ANTHROPIC_API_KEY 환경변수가 설정되지 않았습니다. .env 파일에 추가해주세요.")


In [2]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Claude 모델 초기화
llm = ChatAnthropic(
    model="claude-sonnet-4-5-20250929",
    temperature=0.3,  # 일관된 출력을 위해 낮은 temperature 사용
)


In [ ]:
# 쿼리 변환 프롬프트 템플릿 정의
# SYSTEM_PROMPT = """You are a music query translator. Your task is to convert user's music search query (in any language) into a structured English description format that can be used for music embedding models like CLAP or MuQ-MuLan.

# Output Format (strictly follow this template):
# [genre], [mood] mood, [tempo or energy], [main instruments], [vocal info + language], suitable for [use-case or scene]

# Guidelines:
# 1. Genre: Identify the most likely music genre (e.g., Lo-fi, Electronic, Pop, Jazz, Classical, Ambient, Rock, R&B, Hip-hop, etc.)
# 2. Mood: Describe the emotional atmosphere (e.g., relaxed, energetic, melancholic, happy, romantic, peaceful, intense, etc.)
# 3. Tempo/Energy: Describe the speed or energy level (e.g., slow tempo, medium tempo, fast tempo, calm energy, high energy, etc.)
# 4. Main instruments: List likely instruments (e.g., piano, guitar, synthesizer, drums, strings, etc.) If uncertain, use "soft instruments" or "electronic sounds"
# 5. Vocal info: Specify vocal type or "no vocals" / "instrumental". Include language if vocals are expected.
# 6. Use-case/Scene: Translate or interpret the intended scene or purpose from the query.

# Important:
# - Output ONLY the formatted query string, nothing else.
# - If some information is not clear from the query, make reasonable assumptions based on the context.
# - Keep the output concise and focused on musical characteristics.
# - All output must be in English."""

SYSTEM_PROMPT = """You are a music query translator. Your task is to convert user's music search query (in any language) into a structured English description format that can be used for music embedding models like CLAP or MuQ-MuLan.

Output Format (strictly follow this template):
[genre], [mood] mood, [tempo or energy]

Guidelines:
1. Genre: Identify the most likely music genre
2. Mood: Describe the emotional atmosphere
3. Tempo/Energy: Describe the speed or energy level

Important:
- Output ONLY the formatted query string, nothing else.
- If some information is not clear from the query, make reasonable assumptions based on the context.
- Keep the output concise and focused on musical characteristics.
- All output must be in English."""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "Convert this music search query to the template format: {query}")
])


In [4]:
# LangChain 체인 생성
chain = prompt | llm | StrOutputParser()

def extract_music_query(user_query: str) -> str:
    """
    사용자의 음악 검색 쿼리를 CLAP/MuQ-MuLan 모델에 사용할 수 있는 
    영어 템플릿 형식으로 변환합니다.
    
    Args:
        user_query: 한국어 또는 다른 언어로 된 음악 검색 쿼리
        
    Returns:
        영어 템플릿 형식의 음악 설명 문자열
        
    Example:
        >>> extract_music_query("료칸 여행에 어울리는 숏폼 음악")
        "Ambient, peaceful mood, slow tempo, koto and soft strings, no vocals, 
         suitable for Japanese ryokan travel short-form content"
    """
    result = chain.invoke({"query": user_query})
    return result.strip()


In [5]:
# 테스트: 단일 쿼리 변환
test_query = "료칸 여행에 어울리는 숏폼 음악"
result = extract_music_query(test_query)

print(f"입력 쿼리: {test_query}")
print(f"\n변환 결과:")
print(result)


입력 쿼리: 료칸 여행에 어울리는 숏폼 음악

변환 결과:
Lo-fi, peaceful mood, medium tempo


In [6]:
# 여러 쿼리 테스트
test_queries = [
    "카페에서 공부할 때 듣기 좋은 음악",
    "새벽 드라이브에 어울리는 시티팝",
    "헬스장에서 운동할 때 신나는 음악",
    "비 오는 날 창밖을 보며 듣는 재즈",
    "아침에 일어나서 듣는 상쾌한 팝송",
]

print("=" * 60)
for query in test_queries:
    result = extract_music_query(query)
    print(f"\n🎵 입력: {query}")
    print(f"📝 출력: {result}")
    print("-" * 60)



🎵 입력: 카페에서 공부할 때 듣기 좋은 음악
📝 출력: Lo-fi, relaxed mood, slow tempo
------------------------------------------------------------

🎵 입력: 새벽 드라이브에 어울리는 시티팝
📝 출력: City Pop, nostalgic mood, medium tempo
------------------------------------------------------------

🎵 입력: 헬스장에서 운동할 때 신나는 음악
📝 출력: Electronic, energetic mood, high energy fast tempo
------------------------------------------------------------

🎵 입력: 비 오는 날 창밖을 보며 듣는 재즈
📝 출력: Jazz, melancholic mood, slow tempo
------------------------------------------------------------

🎵 입력: 아침에 일어나서 듣는 상쾌한 팝송
📝 출력: Pop, refreshing mood, medium tempo
------------------------------------------------------------


## 검색 기능 구현

CLAP 모델과 Supabase를 사용하여 추출된 쿼리로 음악을 검색합니다.


In [7]:
from transformers import ClapProcessor, ClapModel
import torch

# CLAP 모델과 processor 로드
processor = ClapProcessor.from_pretrained("laion/larger_clap_music")
clap_model = ClapModel.from_pretrained("laion/larger_clap_music")

print("✓ CLAP 모델 로드 완료")


✓ CLAP 모델 로드 완료


In [ ]:
def get_text_embedding(query: str) -> list:
    """
    텍스트 쿼리를 CLAP 임베딩으로 변환하여 list로 반환
    
    Args:
        query: 음악을 설명하는 텍스트 쿼리
        
    Returns:
        512차원 텍스트 임베딩 벡터 (list)
    """
    inputs = processor(text=query, return_tensors="pt", padding=True)
    
    with torch.no_grad():
        text_emb = clap_model.get_text_features(**inputs)
    
    return text_emb.cpu().numpy().squeeze().tolist()

# 테스트
test_emb = get_text_embedding("electronic music")
print(f"임베딩 길이: {len(test_emb)}")


임베딩 길이: 512


In [9]:
from supabase import create_client

# Supabase 클라이언트 설정
supabase_client = create_client(
    supabase_url=os.getenv("SUPABASE_URL"),
    supabase_key=os.getenv("SUPABASE_SECRET_KEY"),
)

print("✓ Supabase 클라이언트 연결 완료")


✓ Supabase 클라이언트 연결 완료


In [10]:
import IPython.display as ipd

def search_music(query: str, match_count: int = 5, show_audio: bool = True):
    """
    텍스트 쿼리로 음악을 검색합니다.
    
    Args:
        query: 영어로 된 음악 검색 쿼리 (템플릿 형식)
        match_count: 반환할 결과 수
        show_audio: 오디오 플레이어 표시 여부
        
    Returns:
        검색 결과 리스트
    """
    # 쿼리를 CLAP 임베딩으로 변환
    query_embedding = get_text_embedding(query)
    
    # Supabase 벡터 검색 실행
    response = supabase_client.rpc("search_tracks", {
        "query_embedding": query_embedding,
        "match_count": match_count
    }).execute()
    
    results = response.data if hasattr(response, 'data') else response
    
    # 결과 출력
    print(f"🔍 검색 쿼리: {query[:80]}...")
    print("=" * 60)
    
    for i, item in enumerate(results):
        print(f"\n[{i+1}] {item.get('title', 'Unknown')}")
        print(f"    📋 태그: {item.get('tags', '')[:100]}...")
        print(f"    📊 유사도: {item.get('similarity', 0):.6f}")
        
        audio_url = item.get('audioUrl')
        if show_audio and audio_url:
            display(ipd.Audio(audio_url, autoplay=False))
        print("-" * 60)
    
    return results


In [11]:
def search_music_korean(korean_query: str, match_count: int = 5, show_audio: bool = True):
    """
    한국어 쿼리로 음악을 검색합니다.
    내부적으로 LLM을 사용해 영어 템플릿 형식으로 변환 후 검색합니다.
    
    Args:
        korean_query: 한국어로 된 음악 검색 쿼리
        match_count: 반환할 결과 수
        show_audio: 오디오 플레이어 표시 여부
        
    Returns:
        검색 결과 리스트
    """
    print(f"🇰🇷 원본 쿼리: {korean_query}")
    print()
    
    # LLM을 사용해 영어 템플릿 형식으로 변환
    english_query = extract_music_query(korean_query)
    print(f"🇺🇸 변환된 쿼리: {english_query}")
    print()
    
    # 변환된 쿼리로 검색 수행
    return search_music(english_query, match_count, show_audio)


In [17]:
# 테스트: 한국어 쿼리로 음악 검색
results = search_music_korean("료칸 여행에 어울리는 숏폼 음악", match_count=5, show_audio=False)


🇰🇷 원본 쿼리: 료칸 여행에 어울리는 숏폼 음악

🇺🇸 변환된 쿼리: Lo-fi, peaceful mood, medium tempo

🔍 검색 쿼리: Lo-fi, peaceful mood, medium tempo...

[1] 잔향 4
    📋 태그: Indie rock / dance-punk, early-2010s UK indie dance groove; bright jangly palm-muted electric guitar...
    📊 유사도: 0.986724
------------------------------------------------------------

[2] diriring
    📋 태그: jazz hiphop, triplet bounce, catchy simple guitar loop, groovy, sampling sound, vocal chop, 3 minute...
    📊 유사도: 0.990406
------------------------------------------------------------

[3] ambiguous neon
    📋 태그: jazz hip-hop, phonk, sampling sound...
    📊 유사도: 0.991136
------------------------------------------------------------

[4] 어두운 방 3
    📋 태그: indie pop-rock / dream-pop, 115 BPM, clean arpeggiated electric guitar and warm piano chords, airy s...
    📊 유사도: 0.991575
------------------------------------------------------------

[5] Poster 2 UK House ver
    📋 태그: Euphoric pop-house / UK house, 124 BPM, Warm, bubbling synths and si

In [18]:
# 다른 쿼리로 테스트해보기

results = search_music_korean("카페에서 공부할 때 듣기 좋은 음악", show_audio=False)


🇰🇷 원본 쿼리: 카페에서 공부할 때 듣기 좋은 음악

🇺🇸 변환된 쿼리: Lo-fi, relaxed mood, slow tempo

🔍 검색 쿼리: Lo-fi, relaxed mood, slow tempo...

[1] 잔향 4
    📋 태그: Indie rock / dance-punk, early-2010s UK indie dance groove; bright jangly palm-muted electric guitar...
    📊 유사도: 0.986010
------------------------------------------------------------

[2] diriring
    📋 태그: jazz hiphop, triplet bounce, catchy simple guitar loop, groovy, sampling sound, vocal chop, 3 minute...
    📊 유사도: 0.989713
------------------------------------------------------------

[3] ambiguous neon
    📋 태그: jazz hip-hop, phonk, sampling sound...
    📊 유사도: 0.990375
------------------------------------------------------------

[4] 어두운 방 3
    📋 태그: indie pop-rock / dream-pop, 115 BPM, clean arpeggiated electric guitar and warm piano chords, airy s...
    📊 유사도: 0.990891
------------------------------------------------------------

[5] Poster 2 UK House ver
    📋 태그: Euphoric pop-house / UK house, 124 BPM, Warm, bubbling synths and side-c

In [19]:
results = search_music_korean("새벽 드라이브에 어울리는 시티팝", show_audio=False)

🇰🇷 원본 쿼리: 새벽 드라이브에 어울리는 시티팝

🇺🇸 변환된 쿼리: City Pop, nostalgic mood, medium tempo

🔍 검색 쿼리: City Pop, nostalgic mood, medium tempo...

[1] 잔향 4
    📋 태그: Indie rock / dance-punk, early-2010s UK indie dance groove; bright jangly palm-muted electric guitar...
    📊 유사도: 0.984797
------------------------------------------------------------

[2] diriring
    📋 태그: jazz hiphop, triplet bounce, catchy simple guitar loop, groovy, sampling sound, vocal chop, 3 minute...
    📊 유사도: 0.988427
------------------------------------------------------------

[3] ambiguous neon
    📋 태그: jazz hip-hop, phonk, sampling sound...
    📊 유사도: 0.989140
------------------------------------------------------------

[4] 어두운 방 3
    📋 태그: indie pop-rock / dream-pop, 115 BPM, clean arpeggiated electric guitar and warm piano chords, airy s...
    📊 유사도: 0.989416
------------------------------------------------------------

[5] Poster 2 UK House ver
    📋 태그: Euphoric pop-house / UK house, 124 BPM, Warm, bubbling synth

In [20]:
results = search_music_korean("헬스장에서 운동할 때 신나는 음악", show_audio=False)

🇰🇷 원본 쿼리: 헬스장에서 운동할 때 신나는 음악

🇺🇸 변환된 쿼리: Electronic, energetic mood, high energy fast tempo

🔍 검색 쿼리: Electronic, energetic mood, high energy fast tempo...

[1] 잔향 4
    📋 태그: Indie rock / dance-punk, early-2010s UK indie dance groove; bright jangly palm-muted electric guitar...
    📊 유사도: 0.985729
------------------------------------------------------------

[2] diriring
    📋 태그: jazz hiphop, triplet bounce, catchy simple guitar loop, groovy, sampling sound, vocal chop, 3 minute...
    📊 유사도: 0.989018
------------------------------------------------------------

[3] ambiguous neon
    📋 태그: jazz hip-hop, phonk, sampling sound...
    📊 유사도: 0.989646
------------------------------------------------------------

[4] 어두운 방 3
    📋 태그: indie pop-rock / dream-pop, 115 BPM, clean arpeggiated electric guitar and warm piano chords, airy s...
    📊 유사도: 0.989959
------------------------------------------------------------

[5] Poster 2 UK House ver
    📋 태그: Euphoric pop-house / UK house, 124 